In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

# Supported emotion labels (based on RAVDESS or similar datasets)
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Path to your dataset
DATASET_PATH = 'Audio Files/Actor_01'

# Extract features (MFCCs)
def extract_features(file_path):
    try:
        audio, sr = librosa.load(file_path, res_type='kaiser_fast', duration=3, sr=22050, mono=True)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfccs_scaled = np.mean(mfccs.T, axis=0)
        return mfccs_scaled
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Load dataset
def load_data():
    features = []
    labels = []

    for root, _, files in os.walk(DATASET_PATH):
        for file in tqdm(files):
            if file.endswith(('.wav', '.mp3')):
                path = os.path.join(root, file)

                # Extract emotion code from filename (works with RAVDESS-style naming)
                try:
                    emotion_code = file.split('-')[2]
                    emotion = emotion_map.get(emotion_code)
                    if emotion:
                        data = extract_features(path)
                        if data is not None:
                            features.append(data)
                            labels.append(emotion)
                except:
                    continue

    return np.array(features), np.array(labels)

# Load and preprocess data
X, y = load_data()

# Encode labels
le = LabelEncoder()
y_encoded = to_categorical(le.fit_transform(y))

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Build model
model = Sequential([
    Dense(256, input_shape=(40,), activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_encoded.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Predict on test set and show results
predictions = model.predict(X_test)
predicted_labels = le.inverse_transform(np.argmax(predictions, axis=1))
actual_labels = le.inverse_transform(np.argmax(y_test, axis=1))

# Display actual vs predicted
for i in range(len(predicted_labels)):
    print(f"Actual: {actual_labels[i]} | Predicted: {predicted_labels[i]}")
